In [1]:
# Cài OpenJDK 8
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless



# --- Chuyển mặc định sang Java 8 ---
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java 2000
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

# --- Kiểm tra phiên bản ---
!java -version

# --- Export JAVA_HOME để thư viện khác nhận biết ---
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["JVM_PATH"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64/server/libjvm.so"

!echo $JAVA_HOME


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-8-jdk-headless is already the newest version (8u462-ga~us1-0ubuntu2~22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 189 not upgraded.
openjdk version "1.8.0_462"
OpenJDK Runtime Environment (build 1.8.0_462-8u462-ga~us1-0ubuntu2~22.04.2-b08)
OpenJDK 64-Bit Server VM (build 25.462-b08, mixed mode)
/usr/lib/jvm/java-8-openjdk-amd64


In [2]:
## ==============================
# 1️⃣ Copy project zip lên Kaggle và setup
# ==============================

# Hiển thị nội dung input
!ls /kaggle/input/claims-33

# Copy folder/zip từ input về working dir
!cp -r /kaggle/input/claims-33 ./ClaimsExtract33

# Rename folder nếu cần
!mv "./ClaimsExtract33/Claim Extraction" ./ClaimsExtraction33

# Vào thư mục root của project
%cd ./ClaimsExtraction33
!pwd
!ls -l

# ==============================
# 2️⃣ Setup sys.path để import modules
# ==============================
import sys, os

ROOT_DIR = os.getcwd()
print(f"ROOT_DIR: {ROOT_DIR}")

# Thêm ROOT_DIR vào path để import models, utils
if ROOT_DIR not in sys.path:
    sys.path.insert(0, ROOT_DIR)
print("sys.path[0:3]:", sys.path[:3])

!pip install -r requirements.txt
# 5. Cài thư viện cần thiết
!pip install -q transformers datasets evaluate rouge_score sacrebleu

# ==============================
# 3️⃣ Train contextualize model
# ==============================
!python scripts/train_contextualize.py

# Sau khi train xong, folder model sẽ là ví dụ: "vit5-base-contextualize-claim"
MODEL_DIR = os.path.join(ROOT_DIR, "vit5-base-contextualize-claim")
print("Model dir:", MODEL_DIR)
assert os.path.exists(MODEL_DIR), "Folder model không tồn tại!"

# ==============================
# 4️⃣ Download & setup VnCoreNLP
# ==============================
!wget -q https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget -q https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.dict -P vncorenlp/models/wordsegmenter/


# Tạo folder vncorenlp theo đúng cấu trúc
!mkdir -p vncorenlp/models/wordsegmenter
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!mv wordsegmenter.dict vncorenlp/models/wordsegmenter/

VNCORENLP_DIR = os.path.join(ROOT_DIR, "vncorenlp")
print("VnCoreNLP dir:", VNCORENLP_DIR)
assert os.path.exists(VNCORENLP_DIR), "Folder vncorenlp không tồn tại!"

# ==============================
# 5️⃣ Extract claims
# ==============================
!python Test/extract_claims.py

# ==============================
# 6️⃣ Predict contextualized claims
# ==============================
!python Test/predict_contextualize.py

# ==============================
# 7️⃣ Check output
# ==============================
import pandas as pd
contextualized_csv = os.path.join(ROOT_DIR, "Test", "data", "contextualized_claims.csv")
if os.path.exists(contextualized_csv):
    df = pd.read_csv(contextualized_csv)
    print(f"Total claims extracted: {len(df)}")
    display(df.head())
else:
    print("Chưa có file output contextualized_claims.csv")


'Claim Extraction'
/kaggle/working/ClaimsExtraction33
/kaggle/working/ClaimsExtraction33
total 127024
-rw-r--r-- 1 root root      665 Oct  2 10:29 colab_setup.py
-rw-r--r-- 1 root root     4213 Oct  2 10:29 colab_train.py
-rw-r--r-- 1 root root      619 Oct  2 10:29 contextualize_claims.py
-rw-r--r-- 1 root root     4524 Oct  2 10:29 cpu_train_contextualize.py
drwxr-xr-x 2 root root     4096 Oct  2 10:29 data
-rw-r--r-- 1 root root     2364 Oct  2 10:29 extract_claims.py
-rw-r--r-- 1 root root     4489 Oct  2 10:29 kaggle_train_contextualize.py
drwxr-xr-x 2 root root     4096 Oct  2 10:29 models
-rw-r--r-- 1 root root 89026560 Oct  2 10:29 OpenJDK8U-jdk_x64_windows_hotspot_8u462b08.msi
drwxr-xr-x 4 root root     4096 Oct  2 10:29 py_vncorenlp
-rw-r--r-- 1 root root      409 Oct  2 10:29 README.md
-rw-r--r-- 1 root root       77 Oct  2 10:29 requirements.txt
-rw-r--r-- 1 root root 13551616 Oct  2 10:29 rustup-init.exe
drwxr-xr-x 2 root root     4096 Oct  2 10:29 scripts
-rw-r--r-- 1 roo

,ID,claim,article
0,1,Nhưng Nhật Bản vào vòng sau nhờ chỉ số fairpla...,Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...
1,1,"Với tinh thần của những võ sĩ đạo Samurai, như...",Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...
2,1,Đó là lý do đội tuyển áo xanh được mệnh danh S...,Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...
3,1,Các cầu thủ Nhật vui vẻ sau trận thua Ba Lan 0...,Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...
4,1,"Tại World Cup 2018, FIFA sẽ tính điểm fair pla...",Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...
